## Reading HTML file with Langchain and Google

### Installing Required Libraries

In [13]:
#!pip install -U langchain-google-genai
#!pip install -U langchain
#!pip install langchain-community

In [4]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('langchain1_llmapp/api_key.env')) # read local .env file
google_key = os.environ['GOOGLE_API_KEY']

### Loading DEU session 20 of parliament members

##### Cleaning the HTML file

In [11]:
from bs4 import BeautifulSoup
from langchain.docstore.document import Document

file_path = 'data/20_session_DEU.html'
with open(file_path, 'r', encoding='utf-8') as f:
    html = f.read()

soup = BeautifulSoup(html, 'html.parser')

tables = soup.find_all('table')

# Create documents from tables
documents = []
for idx, table in enumerate(tables, start=1):
    table_text = table.get_text(separator=' ', strip=True)
    doc = Document(
        page_content=table_text,
        metadata={"table_index": idx}
    )
    documents.append(doc)

from langchain_community.document_loaders import BSHTMLLoader

file_path = 'data/20_session_DEU.html'
loader = BSHTMLLoader(file_path)
data = loader.load()

print(data)

## Step by step

### Creating an Embedding Model

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

#### Creating an Index

In [ ]:
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.indexes import VectorstoreIndexCreator


vectorstore = DocArrayInMemorySearch.from_documents(documents, embedding_model)

#### Querying the index to pass to the LLM

In [15]:
from langchain_google_genai import ChatGoogleGenerativeAI

query = "List all members of the German Bundestag. Retrieve what you can, it doesn´t have to be perfect. Make it in a table format"
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0)

response = index.query(query, llm = llm)

NameError: name 'index' is not defined

In [19]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

# Initialize the LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)

prompt = ChatPromptTemplate.from_template(
    "List all members of the current German Bundestag in a table format with columns: "
    "Name, Political Party, Constituency. Retrieve what you can, it doesn't have to be perfect."
)

# Create a simple chain
chain = prompt | llm

# Invoke the chain
response = chain.invoke({})

print(response.content)

I cannot directly access and display real-time data like the current members of the German Bundestag.  This kind of information changes frequently (due to elections, resignations, etc.) and requires a live, updated data source.

To get the most up-to-date list, I recommend checking the official website of the German Bundestag (www.bundestag.de).  They usually have a searchable database of their members with the information you're looking for (name, party, constituency).

Alternatively, you could try other reputable sources like German news organizations or parliamentary monitoring websites.  These often publish and maintain lists of current members of parliament.


If you have a static dataset (like a CSV file) of Bundestag members, I can help you format it into a table.  Just provide the data, and I'll do my best to present it clearly.


In [50]:
from IPython.display import display, Markdown

display(Markdown(response))

I can provide a partial list of members, along with some additional information, but the table in the Wikipedia article is incomplete and I cannot generate a full list of all 733 members.  I also cannot reliably extract every name even from the partial table due to formatting inconsistencies.

| Faction/Group | Party | Members (Partial List) |
|---|---|---|
| SPD | SPD | Fahimi, Keller, Mohrs, Maas, Philippi, Kiziltepe, Gremmels, Mansoori, Trăsnea, Grötsch, Mehmet Ali, De Ridder, Rinkert, Bartz, Vontz, Mende, Trăsnea, Lutze, Ruf, Rabanus, Hohmann, Schanbacher |
| CDU/CSU | CDU/CSU | Altmaier, Kramp-Karrenbauer, Storjohann, Hennrich, Schäuble, Berghegger, Jung, Schwarz, Stöcker, Schön, Uhl, Bernstein, Föhr, Pahlmann, Kaufmann, Mannes, Wiesmann, Wellenreuther, Sekmen, Scheuer, Müller |
| Bündnis 90/Die Grünen |  | Krischer, Trittin, Kühn, Stahr, Sekmen, Rottmann, Aeffner, Sacher, von Holtz, Kretz, Krumwiede-Steiner, Kekeritz |
| Die Linke (Gruppe) | Die Linke | Amira Mohamed Ali, Dietmar Bartsch, Susanne Ferschl, Gesine Lötzsch, Nicole Gohlke, Ali Al-Dailami, Jan Korte, Sahra Wagenknecht, Klaus Ernst, Jessica Tatti, Christian Görke, Heidi Reichinnek, Sören Pellmann |
| BSW (Gruppe) |  |  (No names listed in the provided excerpt) |


**Presidium (Leadership):**

* **President:** Bärbel Bas (SPD)
* **Vice Presidents:** Aydan Özoğuz (SPD), Yvonne Magwas (CDU), Katrin Göring-Eckardt (Grüne), Wolfgang Kubicki (FDP), Petra Pau (Linke), *[Open position for AfD]*
* **Elder President (Alterspräsident):** Wolfgang Schäuble (CDU)


This is not a complete list of Bundestag members. The Wikipedia article from which this information is taken is itself incomplete.  It is best to consult the official Bundestag website for a definitive list.